In [7]:
import numpy as np
from PIL import Image
import io

# get image from backend api

def image_cropper(image, object_localisation):

    img = Image.open(io.BytesIO(image))

    # get the dimensions
    img_array = np.asarray(img)
    x_shape = img_array.shape[1]
    y_shape = img_array.shape[0]

    # get the bounding box
    bbox = object_localisation['objects'][0]['vertices']

    # get the bounding box array elements
    for coord in bbox:
        coord['x'] = int(round(coord['x'] * x_shape, 0))
        coord['y'] = int(round(coord['y'] * y_shape, 0))

    # slice out the object image
    obj_array = img_array[bbox[0]['y']:bbox[2]['y'], bbox[0]['x']:bbox[1]['x']]
    pil_img = Image.fromarray(obj_array)
    binary_img = converter_pil_binary(pil_img)

    return pil_img, binary_img

def converter_pil_binary(image: Image):
    img_byte_arr = io.BytesIO()
    image.save(img_byte_arr, format='PNG')
    converted_image = img_byte_arr.getvalue()

    return converted_image


if __name__ == '__main__':
    pass


In [19]:
img = '../test_images/orange_2.jpg'

In [21]:
with open(img, 'rb') as f:
    image_data_bytes = f.read()

In [23]:
from google.cloud import vision

def localize_objects_base64(content):
    """Localize objects in the image

    Args:
    content: The image in base64
    """
    client = vision.ImageAnnotatorClient()
    image = vision.Image(content=content)
    response = convert_response(client.object_localization(image=image).localized_object_annotations)
    return response

def convert_response(objects):
    """
    Convert response from Google Cloud Vision API into a structured format.

    Args:
        objects (List): List of objects detected by Google Cloud Vision API.

    Returns:
        dict: A dictionary containing information about detected objects in the image.
              The dictionary has the following structure:
              {
                  'objects': [
                      {
                          'name': str,  # Name of the detected object
                          'confidence': float,  # Confidence score of the detection
                          'vertices': [  # List of vertices forming a bounding box around the object
                              {
                                  'x': float,  # X-coordinate of the vertex
                                  'y': float   # Y-coordinate of the vertex
                              },
                              ...
                          ]
                      },
                      ...
                  ]
              }
    """
    list_of_objects = []
    for object_ in objects:
        vertices = []
        for vertex in object_.bounding_poly.normalized_vertices:
            vertices.append({'x': vertex.x, 'y':vertex.y})
        obj = {
            'name': object_.name,
            'confidence': object_.score,
            'vertices': vertices
        }
        list_of_objects.append(obj)
    response = {'objects' : list_of_objects}
    return response


In [26]:
object_localisation = localize_objects_base64(image_data_bytes)
object_localisation

{'objects': [{'name': 'Orange',
   'confidence': 0.7698898315429688,
   'vertices': [{'x': 0.3644851744174957, 'y': 0.36910149455070496},
    {'x': 0.6173803210258484, 'y': 0.36910149455070496},
    {'x': 0.6173803210258484, 'y': 0.7042608261108398},
    {'x': 0.3644851744174957, 'y': 0.7042608261108398}]},
  {'name': 'Packaged goods',
   'confidence': 0.5047340393066406,
   'vertices': [{'x': 0.1538885235786438, 'y': 0.8863508701324463},
    {'x': 0.5678027272224426, 'y': 0.8863508701324463},
    {'x': 0.5678027272224426, 'y': 0.9973958134651184},
    {'x': 0.1538885235786438, 'y': 0.9973958134651184}]}]}

In [28]:
pil_img, bin_img = image_cropper(image_data_bytes, object_localisation)

SystemError: tile cannot extend outside image